# Praktiske Anvendelser av Rekker med Python

Når du lærer om teorien om rekker i matematikk R2 og S2, er praktiske anvendelser en sentral del av teorien. I denne seksjonen skal vi se på hvordan vi kan løse praktiske problemer ved hjelp av programmering av rekkene.

Vi skal se på:
1. Medikamenter i blodet
2. Annuitetslån



## Medikamenter i blodet

En typisk problemstilling handler om at man regelmessig tar en dose $M_0$ av et medikament som brytes ned i kroppen med en fast nedbrytningsrate. Vi kan i praksis finne en formel for dette å evaluere denne formelen for å finne ut hvor mye medikament som er i blodet til enhver tid.

### Eksempel 1

Tenk deg at du tar en dose $M_0 = 400 \ \text{mg}$ med Paracet én gang i døgnet. Per time brytes det ned 29 % av virkestoffet i blodplasma. Hva er den største mengden virkestoff du har i kroppen over lang tid? 

Virkestoffet brytes ned med 29 % per time, som gir en vekstfaktor $V = 1 - 0.29 = 0.71$ per time. En dose vil da ha en vekstfaktor på $V^24$ per døgn (fordi det 24 timer i døgnet). En strategi for å regne ut virkestoffet i kroppen over tid vises i {prf:ref}`algo-paracet-daglig-dose`.

```{prf:algorithm} Nedbrytning av virkestoff per døgn
:label: algo-paracet-daglig-dose

__Input__: Daglig dose $M_0$, nedbrytning i prosent per time $r$, antall døgn $n$.

__Output__: Mengde virkestoff i kroppen $M$ etter $n$ døgn.


- Sett $M = M_0$
- Sett $V = 1 - r/100$ 
- Sett $k = V^{24}$ 
- For $i = 1, 2, \ldots, n$:
    - Sett $M = k \cdot M$ *(Nedbrytning av virkestoff)*
    - Sett $M = M + M_0$  *(Ny dose)*    
- Returner $M$

```

Vi kan implementere algoritmen med en Pythonkode slik:

In [7]:
M_0 = 400 # daglig dose i mg.
r = 29 # Nedbrytning i prosent per time.
vekstfaktor = 1 - r/100 # vekstfaktor per time
k = vekstfaktor**24 # vekstfaktor per døgn
n = 100 # Antall døgn

M = M_0 # startdose ved første døgn
for _ in range(n):
    M = M * k # nedbrytning av stoffet over ett døgn
    M = M + M_0 # Ny dose legges til virkestoffet i kroppen

print(f"{M = :.2f} mg.") # skriver ut med 2 desimaler

M = 400.11 mg.


### Eksempel 2: En dose av paracet hver 6.time

Et mer realistisk eksempel er at man tar to tabletter med paracet hver 6.time som tilsvarer 1000 mg. Nedbrytningen per time er på 29% av dosen. 
Hva er den største mengden virkestoff du har i kroppen over lang tid?

Vi kan modifisere algoritmen litt for å løse dette problemet, som vises i {prf:ref}`algo-paracet-two`.

```{prf:algorithm} Nedbrytning av Paracet ved inntak hver 6.time.
:label: algo-paracet-two

__Input__: Dose $M_0$ hver 6.time, nedbrytning i prosent per time $r$, antall døgn $n$.

__Output__: Mengde virkestoff i kroppen $M$ etter $n$ døgn.


- Sett $M = M_0$
- Sett $V = 1 - r/100$ 
- Sett $k = V^6$ *(Vekstfaktor for nedbrytning per 6.time)* 
- Sett $M = M_0$ *(Første dose)*
- Sett $M_\text{max} = M$ *(Høyeste dose så langt)*. 
- For $i = 1, 2, \ldots, n$:
    - Sett $M = k \cdot M$ *(Nedbrytning av virkestoff)*
    - Sett $M = M + M_0$  *(Ny dose)*    
    - If $M > M_\text{max}$: *(Hvis dosen er nå høyere enn forrige maksdose)*
        - Sett $M_\text{max} = M$ *(Sett ny maksdose)*
- Returner $M_\text{max}$
```

Vi kan regne ut dette med følgende kode:

In [8]:
M0 = 1000 # mg dose per 6.time
r = 29 # Nedbrytning i prosent per time.
vekstfaktor = 1 - r/100 # vekstfaktor per time
k = vekstfaktor**6 # vekstfaktor per 6.time
n = 100 # Antall 6.timer

M = M0 # startdose ved første 6.time
M_max = M 
for _ in range(n):
    M = M * k # nedbrytning av stoffet over ett 6.time
    M = M + M0 # Ny dose legges til virkestoffet i kroppen
    M_max = max(M, M_max) # henter ut den største verdien av `M` og `M_max`.

print(f"{M_max = :.2f} mg.") # skriver ut med 2 desimaler

M_max = 1146.92 mg.


## Nedbetaling av lån

En annen typisk anvendelse av rekker er nedbetaling av lån. Vi kan formulere en algoritme for dette som vises i {prf:ref}`algo-annuitetslaan`.

```{prf:algorithm} Nedbetaling av lån
:label: algo-annuitetslaan

__Input__: Lånebeløp $L_0$, rente $r$, antall terminer $n$, terminbeløp $x$.

__Output__: Gjenstående lån $L_0$ etter $n$ terminer.


- Sett $L = L_0$
- Sett $V = 1 + r/100$
- `for` $i = 1, 2, \ldots, n$:
    - Sett $L = L\cdot V$ *(Lånet øker med renten)*
    - Sett $L = L - x$ *(Lånet reduseres med terminbeløpet)*
- `return` $L$ *(Gjenstående lån)*

```

Målet vårt er å se på hvordan vi kan bestemme hvilket terminbeløp vi skal betale ved å bruke {prf:ref}`algo-annuitetslaan`. Vi kan gjøre dette ved å løse likningen $L = 0$ for $x$. Og selv om vi har sett på effektive metoder for å gjøre dette i [Finne nullpunkter numerisk](../nullpunkter/intro.md), så skal vi først se på et eksempel der vi har en *naiv* implementasjon av algoritmen.


```{admonition} En naiv algoritme, hva er det?
:class: tip, dropdown

En *naiv* algoritme er en løser problemet med *brute force*, og vil løse problemet, men er ikke spesielt effektiv og vil ofte være ganske treig.

```

````{admonition} Sammenheng mellom algoritmen og summeformler for rekker anvendt på annuitetslån
:class: tip, dropdown

Hvis vi følger algoritmen her og antar av vi skal betale ned lånet på $n$ terminer, så vil vi få følgende sammenhenger:

\begin{align*}
L_1 & = L_0V - x, \\
L_2 & = L_1V - x = L_0V^2 - xV - x, \\
L_3 & = L_2V - x = L_0V^3 - xV^2 - xV - x, \\
\vdots & \qquad \qquad \qquad \vdots \qquad \qquad \qquad  \vdots & \\
L_n & = L_0V^n - xV^{n-1} - xV^{n-2} - \ldots - xV^2 - xV - x.
\end{align*}

Typisk er målet etter $n$ terminer at restlånet skal være null slik at $L_n = 0$. Vi kan da løse ut for $x$ og få følgende formel for terminbeløpet:

$$
0 = L_0V^n - xV^{n-1} - xV^{n-2} - \ldots - xV^2 - xV - x,
$$

som gir

$$
x + xV + xV^2 + \ldots + xV^{n-2} + xV^{n-1} = L_0V^n.
$$

som er det man ofte kaller for å bruke *sluttverdier*. Og hvis vi deler med $V^n$ i alle ledd får vi

$$
\frac{x}{V} + \frac{x}{V^2} + \frac{x}{V^3} + \ldots + \frac{x}{V^{n-1}} + \frac{x}{V^n} = L_0,
$$

som er da vi bruker *nåverdier*. 

Konklusjonen er at algoritmen i {prf:ref}`algo-annuitetslaan` er mer generell og gir oss restlån etter $n$ terminer med et vilkårlig terminbeløp $x$.

````

### Eksempel 1: Finne restlån etter $n$ terminer

Anta vi tar et lån på 1 million kroner med en rente på 3% per år. Vi velger å betale ned lånet med et terminbeløp på 70 000 kr over 10 år. Hva er da restlånet etter 10 år? 

Først kan vi systematisere informasjonen litt, sånn at det er i tråd med notasjonen i {prf:ref}`algo-annuitetslaan`. Vi har

- Lånetbeløpet $L_0 = 1 000 000 \text{ kr}$.
- Renten $r = 3\%$.
- Antall terminer $n = 10$.
- Terminbeløp $x = 70 000 \text{ kr}$.

Vi kan da bruke algoritmen i {prf:ref}`algo-annuitetslaan` til å finne restlånet etter 10 år:

In [1]:
lån = 1_000_000 # kroner
rente = 3 # 3 prosent rente per år
vekstfaktor = 1 + rente / 100 # vekstfaktor per år
terminbeløp = 70_000 # Terminbeløp per år
n_terminer = 10 # antall terminer (her: antall år)

for _ in range(n_terminer):
    lån = lån * vekstfaktor
    lån = lån - terminbeløp

print(f"{lån = :.2f} kr gjenstår etter {n_terminer} år.")

lån = 541444.83 kr gjenstår etter 10 år.


Så det gjenstår altså et lån ca. 541 445 kr etter 10 år med et terminbeløp på 70 000 kr.

### Eksempel 2: Finne terminbeløp for å betale ned lånet på 10 år

La oss ta utgangspunkt i samme lånesituasjon, men nå skal vi finne hvilket terminbeløp $x$ vi må betale for å betale ned lånet på 10 år. Vi kan da bruke algoritmen i {prf:ref}`algo-annuitetslaan` til å finne terminbeløpet. 

Men denne gangen, så justerer vi algoritmen litt for å finne terminbeløpet $x$:

````{prf:algorithm} Finne terminbeløp for å betale ned lånet på $n$ terminer
:label: algo-annuitetslaan-belop

__Input__: Lånebeløp $L_0$, rente $r$, antall terminer $n$, endring av terminbeløp $\Delta x$ mellom forsøk.

__Output__: Terminbeløpet $x$ som gjør at lånet er nedbetalt etter $n$ terminer.


- Sett $V = 1 + r/100$ *(Vekstfaktoren til renten)*
- Sett $x = 0$ *(Vi starter med å sette terminbeløpet til 0)*
- `while` $L > 0$: *(Så lenge restlånet er større enn 0 etter et nedbetalingforsøk)*
    - Sett $L = L_0$ *(Setter restlånet lik opprinnelig ved hvert forsøk)*
    - Sett $x = x + \Delta x$ *(Øker terminbeløpet med $\Delta x$)*
    - Sett $V = 1 + r/100$
    - `for` $i = 1, 2, \ldots, n$:
        - Sett $L = L\cdot V$ *(Lånet øker med renten)*
        - Sett $L = L - x$ *(Lånet reduseres med terminbeløpet)*
- `return` $x$ *(Returnerer terminbeløpet som betaler ned hele lånet)*
````

Følger vi {prf:ref}`algo-annuitetslaan-belop`, kan vi skrive koden slik:

In [8]:
lån = 1e6 # 1 000 000 kroner
rente = 3 # 3 prosent rente per år
vekstfaktor = 1 + rente / 100 # vekstfaktor per år

terminbeløp = 0
dx = 0.1 # 100 kroner økning i terminbeløp per år
n_terminer = 10
restlån = lån

while restlån > 0:
    terminbeløp += dx
    restlån = lån
    for _ in range(n_terminer):
        restlån = restlån * vekstfaktor
        restlån = restlån - terminbeløp
    
print(f"{terminbeløp = :.2f} kr per år.")

terminbeløp = 117230.60 kr per år.


som gir at hvis vi setter terminbeløpet $x = 117230.60 \text{ kr}$, så vil lånet være nedbetalt etter 10 år. Dette er bare en tilnærming til det *sanne* terminbeløpet siden vi har brukt $\Delta x = 0.01 \ \text{kr}$ mellom hvert forsøk. Men det er en god nok tilnærming med tanke på størrelsen på terminbeløpet. En endring på 0.01 kr er ikke merkbar i forhold til et terminbeløp på $117 230.60$ kr.

## Oppgaver

### Oppgave 1

Du tar opp et lån på 3 millioner kroner. Renten på lånet er på 2.5 % per år. Terminbeløpet du skal betale per termin (per år) er 130 000 kroner. Bruk Python til å finne ut hvor mange år det tar før du har betalt ned hele lånet.

*Du kan ta utgangspunkt i kodeskallet under. Du må fylle inn der det står `NotImplemented`.*

In [ ]:
lån = NotImplemented
rente = NotImplemented
vekstfaktor = NotImplemented
terminbeløp = NotImplemented

år = 0
while NotImplemented: # Sett en betingelse for når løkken skal stoppe
    lån = NotImplemented
    år = NotImplemented

print(f"{år = }")

````{dropdown} Løsningsforslag

```python
lån = 3_000_000
rente = 2.5
vekstfaktor = 1 - rente / 100
terminbeløp = 130_000

år = 0
while lån > 0: # Sett en betingelse for når løkken skal stoppe
    lån = lån * vekstfaktor - terminbeløp
    år = år + 1

print(f"{år = }")
print(f"gjenstående {lån = :.2f} kr")
```

som gir utskriften

```console
år = 18
gjenstående lån = -1273.92 kr
```

````